In [1]:
import sys
import os
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'white')
# sns.set(style='whitegrid', color_codes=True)
from tqdm import tqdm
import plotly
import time
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo
import warnings
import random
import gc
# import dask.dataframe as dd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import validation
import re
from sys import getsizeof
import scipy
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from itertools import cycle
from pandas import Grouper
from scipy.sparse import csr_matrix
import joblib
from sklearn import preprocessing
import lightgbm as lgbm
from sklearn import metrics
import joblib
from sklearn.preprocessing import OneHotEncoder
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Flatten,BatchNormalization,Dropout,LeakyReLU,LSTM,Conv1D,TimeDistributed,MaxPool1D,Flatten,Input
from tensorflow.keras.regularizers import l1,l2,l1_l2
import tensorflow.keras.initializers as initializer
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
import json
tf.random.set_seed(120)
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
# rfered this blog
# https://pythonhosted.org/calmap/
# np.random.seed(sum(map(ord, 'calmap')))
import calmap
import pickle
import joblib
# from joblib import Parallel,delayed
from scipy.stats import norm
from sklearn.feature_extraction.text import TfidfVectorizer
color_cycle = cycle(["blue","green","red","cyan","magenta",'yellow','black','darkorange','grey','lime'])
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
tf.executing_eagerly() 

True

In [132]:
def reduce_mem_usage(df):
  incoming_df_mem_usage = df.memory_usage().sum() / 1024**2
#   print("Memory usage of properties dataframe is :",incoming_df_mem_usage," MB")
  na_list = [] # to keep track the columns that have missing values filled in
  for col in df.columns:
    if df[col].dtype != object:
      is_int = False
      max =  df[col].max()
      min = df[col].min()

      # integer doesn't support nan, so it needs to be filled
      if not np.isfinite(df[col].all()):
        na_list.app(col)
        df[col].fillna(mn-1,inplace=True)
      # test if column can be converted to integer
      asint = df[col].fillna(0).astype(np.int64)
      result = (df[col] - asint)
      result = result.sum()

      if result > -0.01 and result < 0.01:
        is_int = True
      if is_int:
        if min>=0:
          if max < 255:
            df[col] = df[col].astype(np.uint8)
          elif max < 65535:
            df[col] = df[col].astype(np.uint16)
          elif max < 4294967295:
            df[col] = df[col].astype(np.uint32)
          else:
            df[col] = df[col].astype(np.uint64)
        else:
          if min > np.iinfo(np.int8).min and max < np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
          elif min > np.iinfo(np.int16).min and max < np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
          elif min > np.iinfo(np.int32).min and max < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
          elif min > np.iinfo(np.int64).min and max < np.iinfo(np.int64).max:
            df[col] = df[col].astype(np.int64)
      else:
        df[col] = df[col].astype(np.float32)
         
#   print("********************** MEMORY USAGE AFTER COMPLETION **********************")
  outgoing_df_mem_usage = df.memory_usage().sum() / 1024**2 
#   print("Memory usage is: ",outgoing_df_mem_usage," MB")
#   print("This is ",100*outgoing_df_mem_usage/incoming_df_mem_usage,"% of the initial size")
  return df, na_list

**We have data for 1941 days, first 1913 for train dataset and 1914-1941 is validation dataset and we have to predict the sales for next 28 days so, we have add d_42 to d_1970 with values and make it as our test dataset**

**Read the below instructions before giving the input data to predict.**
1. You have sales data information from 0 to 1942 and we have calender(for example event information and snap data information)
   and we have sell price information(price of a product over time) till 1969, so we have predict till 1969 days only
2. from the user input perspective the function need two inputs one is for what products that user want to see the predicted
   output and for what are the days user can give range of days or specific days.

In [141]:
def function1(input_data_points,pred_day):
    pred_day.sort()
    raw_input_data = {'id':input_data_points,'item_id':[],'dept_id':[],'cat_id':[],'store_id':[],'state_id':[]}
    for data_point in input_data_points:
        data_point = data_point.split('_')
        raw_input_data['item_id'].append(f'{data_point[0]}_{data_point[1]}_{data_point[2]}')
        raw_input_data['dept_id'].append(f'{data_point[0]}_{data_point[1]}')
        raw_input_data['cat_id'].append(f'{data_point[0]}')
        raw_input_data['store_id'].append(f'{data_point[3]}_{data_point[4]}')
        raw_input_data['state_id'].append(f'{data_point[3]}')
        
        
    sales_train_evaluation = pd.read_pickle(r"best_model/sales_train_evaluation.pkl")  #The properties dataset
    calendar = pd.read_pickle(r"best_model/calender.pkl")  #The properties dataset
    sell_prices = pd.read_pickle(r"best_model/sell_prices.pkl")  #The properties dataset

    if(len(pred_day)!=1):
        start_day,end_day = min(pred_day),max(pred_day)
    else:
        start_day,end_day = pred_day[0],pred_day[0]

    sales_train_evaluation_ip = pd.DataFrame.from_dict(raw_input_data)

    sales_train_evaluation_col = [column for column in sales_train_evaluation.columns if 'd_' not in column]

    day_range_to_predict = [f'd_{day}' for day in range(start_day,end_day+1)]

    test_day = [f'd_{day}' for day in range(1942,1970)]

    d_cols = []
    for d_num in range(start_day - 90,start_day):
        d_cols.append(f'd_{d_num}')
    d_cols = d_cols + day_range_to_predict
    days_from_test_data = list(set(d_cols).intersection(test_day))

    for d in days_from_test_data:
        sales_train_evaluation_ip[d] = 0
        sales_train_evaluation_ip[d] = sales_train_evaluation_ip[d].astype(np.int16)

    sales_train_evaluation_ip_cols_filled = [d for d in sales_train_evaluation_ip.columns if 'd_' in d]
    days_from_rest = set(sales_train_evaluation_ip_cols_filled) ^ set(d_cols)
    items_to_select = sales_train_evaluation_ip[['id']]
    sales_train_selected_item = pd.merge(sales_train_evaluation,items_to_select,on='id',how='inner')
    sales_train_evaluation_d_columns = sales_train_selected_item[days_from_rest]
    sales_train_evaluation_ip = pd.concat([sales_train_evaluation_ip,sales_train_evaluation_d_columns],axis=1)
    sales_train_evaluation_ip_cols_cat,sales_train_evaluation_ip_cols_d = sales_train_evaluation_col,[d for d in list(sales_train_evaluation_ip.columns) if 'd_' in d]
    sales_train_evaluation_ip = sales_train_evaluation_ip[sales_train_evaluation_ip_cols_cat + list(np.sort(np.asarray(sales_train_evaluation_ip_cols_d)))]
    
    day_cols_calender = [column for column in sales_train_evaluation_ip.columns if 'd_' in column]
    calendar_ip = calendar[calendar['d'].isin(day_cols_calender)]
    sell_prices_ip = sell_prices
    
    
    df_ip = pd.melt(sales_train_evaluation_ip, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='units_sold').dropna()
    df_ip = pd.merge(df_ip, calendar_ip, on='d', how='left')
    df_ip = pd.merge(df_ip, sell_prices_ip, on=['store_id','item_id','wm_yr_wk'], how='left') 
    
    df_ip['date'] = pd.to_datetime(df_ip['date'])
    df_ip['day_of_month'] = df_ip['date'].dt.day.astype('int8')
    df_ip['week_of_month'] = df_ip['date'].apply(lambda d: (d.day-1) // 7 + 1).astype('int8')
    df_ip['week_of_year'] = df_ip['date'].dt.week.astype('int8')
    df_ip['quarter_of_year'] = df_ip['date'].dt.quarter.astype('int8')
    
    df_ip['is_month_start'] = [1 if i == 1 else 0 for i in df_ip['week_of_month']]
    df_ip['is_month_end'] = [1 if i in [4,5] else 0 for i in df_ip['week_of_month']]
    df_ip['event_name_1'] = df_ip['event_name_1'].fillna(0)
    df_ip['event_name_2'] = df_ip['event_name_2'].fillna(0)
    df_ip['event_type_1'] = df_ip['event_type_1'].fillna(0)
    df_ip['event_type_2'] = df_ip['event_type_2'].fillna(0)
    df_ip['is_week_end'] = [1 if i in [1,2] else 0 for i in df_ip['wday']]
    df_ip['close_to_week_days'] = [1 if i in [3,7] else 0 for i in df_ip['wday']]
    df_ip['mid_weeK_days'] = [1 if i in [4,5,6] else 0 for i in df_ip['wday']]
    df_ip['is_weekday'] = [1 if i not in [1,2] else 0 for i in df_ip['wday']]
    
    df_ip = df_ip.drop(['date','wm_yr_wk','weekday'],axis=1)

    avg_store_item_df = pd.read_pickle('best_model/preprocessed/avg_store_item_df_unique.pkl')
    avg_store_item_df = avg_store_item_df.drop_duplicates()
    
    df_ip = pd.merge(df_ip,avg_store_item_df,left_on=['store_id','item_id'],right_on=['store_id','item_id'])
    
    item_id_le = joblib.load('best_model/preprocessed/item_id_le.pkl')
    dept_id_le = joblib.load('best_model/preprocessed/dept_id_le.pkl')
    cat_id_le = joblib.load('best_model/preprocessed/cat_id_le.pkl')
    state_id_le = joblib.load('best_model/preprocessed/state_id_le.pkl')
    store_id_le = joblib.load('best_model/preprocessed/store_id_le.pkl')
    event_name_1_le = joblib.load('best_model/preprocessed/event_name_1_le.pkl')
    event_name_2_le = joblib.load('best_model/preprocessed/event_name_2_le.pkl')
    event_type_1_le = joblib.load('best_model/preprocessed/event_type_1_le.pkl')
    event_type_2_le = joblib.load('best_model/preprocessed/event_type_2_le.pkl')

    df_ip['item_id'] = item_id_le.transform(df_ip['item_id'].values)
    df_ip['dept_id'] = dept_id_le.transform(df_ip['dept_id'].values)
    df_ip['cat_id'] = cat_id_le.transform(df_ip['cat_id'].values)
    df_ip['state_id'] = state_id_le.transform(df_ip['state_id'].values)
    df_ip['store_id'] = store_id_le.transform(df_ip['store_id'].values)

    cat_df_ip = reduce_mem_usage(df_ip[['item_id','dept_id','cat_id','state_id','store_id']])
    df_ip = df_ip.drop(['item_id','dept_id','cat_id','state_id','store_id'],axis=1)

    df_ip['event_name_1'] = event_name_1_le.transform(df_ip['event_name_1'].astype(str).values)
    df_ip['event_name_2'] = event_name_2_le.transform(df_ip['event_name_2'].astype(str).values)
    df_ip['event_type_1'] = event_type_1_le.transform(df_ip['event_type_1'].astype(str).values)
    df_ip['event_type_2'] = event_type_2_le.transform(df_ip['event_type_2'].astype(str).values)

    event_df_ip = reduce_mem_usage(df_ip[['event_name_1','event_name_2','event_type_1','event_type_2']])
    df_ip = df_ip.drop(['event_name_1','event_name_2','event_type_1','event_type_2'],axis=1)
    df_test_ip = pd.concat([cat_df_ip[0],event_df_ip[0]],axis=1) 
    df_ip = pd.concat([df_ip,df_test_ip],axis=1)
    del cat_df_ip
    del event_df_ip
    del df_test_ip
    gc.collect()
    
    for lag_day,lag_column in zip([7,28,29,30,31,90],[f"lag_{lag}" for lag in [7,28,29,30,31,90]]):
        df_ip[lag_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].shift(lag_day).astype(np.float16)

    for rolling_day,rolling_column in zip([7,28,29,30,31,90],[f"rolling_{rolling}" for rolling in [7,28,29,30,31,90]]):
        df_ip[rolling_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].transform(lambda x: x.rolling(window=rolling_day).mean()).astype(np.float16)

    for rolling_day,rolling_column in zip([7,28,29,30,31,90],[f"rolling_std_{rolling}" for rolling in [7,28,29,30,31,90]]):
        df_ip[rolling_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].transform(lambda x: x.rolling(window=rolling_day).std()).astype(np.float16)

    for expanding_day,expanding_column in zip([7,28,29,30,31,90],[f"expanding_{expanding}" for expanding in [7,28,29,30,31,90]]):
        df_ip[expanding_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].transform(lambda x: x.expanding(expanding_day).mean()).astype(np.float16)

    for expanding_day,expanding_column in zip([7,28,29,30,31,90],[f"expanding_std_{expanding}" for expanding in [7,28,29,30,31,90]]):
        df_ip[expanding_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].transform(lambda x: x.expanding(expanding_day).std()).astype(np.float16)
        
    df_ip['d'] = df_ip['d'].apply(lambda x:x.split('_')[1]).astype(np.int16)
    df_ip['store_id'] = [str(i.split('_')[-3])+'_'+str(i.split('_')[-2]) for i in df_ip['id']]
    df_ip['units_sold'] = df_ip['units_sold'].fillna(0)
    
    with open('best_model/preprocessed/other_info.json') as f:
        other_info = json.load(f)

    df_ip['sell_price'] =  df_ip['sell_price'].fillna(other_info['median_sell_price'])
    df_ip['daily_avg_sales_unit'] = df_ip.groupby(['id','d'])['units_sold'].transform('mean').astype(np.float16)
    df_ip['weekly_avg_sales_unit'] = df_ip.groupby(['id','week_of_month'])['units_sold'].transform('mean').astype(np.float16)
    df_ip['sales_usd'] = df_ip['sell_price'] * df_ip['units_sold']
    
    final_value = {}
    final_value['valid_pred'] = {}
    final_value['test_pred'] = {}
    remove_columns = ['id','store_id','sales_usd','daily_avg_sales_unit','is_weekday','units_sold']
    store_list = df_ip['store_id'].unique()
    for store in store_list:
        df_store = df_ip[df_ip['store_id'] == store]  
        snap_feature  = ['snap_'+store.split('_')[0]]
        selected_columns = [column for column in df_store.columns if '7' not in column and 'rolling' not in column and 'snap' not in column and 'state' not in column]
        df_store = df_store[selected_columns+snap_feature]
        print(df_store.shape)
        x_test = df_store.loc[(df_store['d']>=pred_day[0])].drop(remove_columns,axis=1)
        print('*****Prediction for Store: {}*****'.format(store))  
        dummy_model = joblib.load(f'best_model/gbdt/{store}_median_model1.pkl')
        test_pred_df = pd.DataFrame()
        #     val_score = np.sqrt(metrics.mean_squared_error(val_pred,y_valid))
        y_pred = dummy_model.predict(x_test)
        y_pred = [round(val) for val in y_pred]
        test_pred_df['id'] = df_store.loc[(df_store['d']>=pred_day[0])]['id']
        test_pred_df['store_id'] = df_store.loc[(df_store['d']>=pred_day[0])]['d']
        test_pred_df['units_sold'] = y_pred
        final_value['test_pred'][store] = test_pred_df

    dict_store_test = {}
    dict_store_test['id'] = []
    dict_store_test['d'] = []
    dict_store_test['units_sold'] = []
    for k,v in final_value['test_pred'].items():
        for i in v.values:
            dict_store_test['id'].append(i[0])
            dict_store_test['d'].append(i[1])
            dict_store_test['units_sold'].append(i[2])

    test_pred = pd.DataFrame.from_dict(dict_store_test)
    test_pred = pd.pivot(test_pred, index = 'id', columns = 'd', values = 'units_sold').reset_index()
    test_pred.columns = ['id'] + ['d_' + str(i) for i in test_pred.columns if 'id' not in str(i)]
    test_pred = test_pred[['id']+['d_'+str(i) for i in pred_day]]
    return test_pred

In [142]:
# # if we want test with multiple products / days uncomment and test the following code
# start_time = time.time()
# input_data_points = ['FOODS_3_821_WI_3_evaluation','FOODS_3_822_WI_3_evaluation','FOODS_3_823_WI_3_evaluation']
# days_to_predict = [day for day in range(1913,1941)]
# predicted_data = function1(input_data_points,days_to_predict)
# print("running time {:.3f} secs...".format(time.time() - start_time))
# predicted_data

In [143]:
start_time = time.time()
input_data_points = ['FOODS_3_821_WI_3_evaluation']
days_to_predict = [1920]
predicted_data = function1(input_data_points,days_to_predict)
print("running time {:.3f} secs...".format(time.time() - start_time))
predicted_data

(91, 45)
*****Prediction for Store: WI_3*****
running time 4.447 secs...


,id,d_1920
0,FOODS_3_821_WI_3_evaluation,1.0


In [42]:
def level_aggregator(inp_df):
    roll_mat_csr_store = {}
    stores_list = inp_df['store_id'].unique()
    for store in stores_list:
        store_df = inp_df[inp_df['store_id']==store]
        dummies_list = [store_df.state_id, store_df.store_id, 
                    store_df.cat_id, store_df.dept_id, 
                    store_df.state_id +'_'+ store_df.cat_id, store_df.state_id +'_'+ store_df.dept_id,
                    store_df.store_id +'_'+ store_df.cat_id, store_df.store_id +'_'+ store_df.dept_id, 
                    store_df.item_id, store_df.state_id +'_'+ store_df.item_id, store_df.id]

        # Level_0 aggregation of all values:
        dummy_df =[pd.DataFrame(np.ones(store_df.shape[0]).astype(np.int8), 
                                       index=store_df.index, columns=['all']).T]
        # List of dummy dataframes:
        for i, cats in enumerate(dummies_list):
            dummy_df +=[pd.get_dummies(cats, drop_first=False, dtype=np.int8).T]

        # Concat dummy dataframes in one go:
        roll_mat_df = pd.concat(dummy_df, keys=list(range(12)), 
                                names=['level','id'])#.astype(np.int8, copy=False)

        roll_index = roll_mat_df.index
        roll_mat_csr = csr_matrix(roll_mat_df.values)
        roll_mat_csr_store[store] = roll_mat_csr
    return roll_mat_csr_store

In [43]:
# Fucntion to calculate S weights for each store:
def get_s(df_ip,day_range,roll_mat_csr_store,drop_days=0):
    weight_s_list = {}
    stores_list = df_ip['store_id'].unique()
    for store in stores_list:    
        """
        drop_days: int, equals 0 by default, so S is calculated on all data.
                   If equals 28, last 28 days won't be used in calculating S.
        """
        data_store = df_ip[df_ip['store_id']==store]
        roll_mat_csr = roll_mat_csr_store[store]
        # Rollup sales:
#         d_name = ['d_' + str(i) for i in range((day_range[0]-90),day_range[1]+1)]
#         print(d_name)
#         print(roll_mat_csr.shape,data_store[d_name].values.shape)
        d_name = [d for d in df_ip.columns if 'd_' in d]
        sales_train_val = roll_mat_csr * data_store[d_name].values

        no_sales = np.cumsum(sales_train_val, axis=1) == 0
        sales_train_val = np.where(no_sales, np.nan, sales_train_val)

        # Denominator of RMSSE 
        weight1 = np.nanmean(np.diff(sales_train_val,axis=1)**2,axis=1)
        weight_s_list[store] = weight1
    return weight_s_list

In [44]:
# Functinon to calculate weights:
def get_w(sale_usd,roll_mat_csr_store,store):
    # calulate sales in USD for each id:
    total_sales_usd = sale_usd.groupby(
        ['id'], sort=False)['sales_usd'].apply(np.sum).values
    
    roll_mat_csr = roll_mat_csr_store[store]
    # Roll up total sales by ids to higher levels:
    weight2 = roll_mat_csr * total_sales_usd
    return 12*weight2/np.sum(weight2)

In [45]:
#Since WRMSSE calucated for each stores so we have 3049 rows and 9180 time series
# Function to do quick rollups:
store_name = ''
def rollup(v,store,roll_mat_csr_store):
    '''
    v - np.array of size (3049 rows, n day columns)
    v_rolledup - array of size (n, 9180)
    '''
    roll_mat_csr = roll_mat_csr_store[store]
    return roll_mat_csr*v #(v.T*roll_mat_csr.T).T


# Function to calculate WRMSSE:
key = 0
def wrmsse(preds, y_true,store_name,roll_mat_csr_store,SW_store,num_of_items,num_of_days):
    '''
    preds - Predictions: pd.DataFrame of size (3049 rows, N day columns)
    y_true - True values: pd.DataFrame of size (3049 rows, N day columns)
    sequence_length - np.array of size (9180,)
    sales_weight - sales weights based on last 28 days: np.array (9180,)
    '''
    preds = preds[-(num_of_items * num_of_days):]
    y_true = y_true[-(num_of_items * num_of_days):]
    preds = preds.reshape(num_of_days, num_of_items).T
    y_true = y_true.reshape(num_of_days, num_of_items).T  
    final_wrmsse = np.sqrt(np.mean(np.square(rollup(preds-y_true,store_name,roll_mat_csr_store)),axis=1)) * SW_store
#     final_wrmsse = [round(val,4) for val in final_wrmsse]
    final_wrmsse = np.nan_to_num(final_wrmsse)
    final_wrmsse = np.asarray([round(val,4) for val in final_wrmsse])
#     print(np.sum(final_wrmsse))
    return 'wrmsse', round(np.sum(final_wrmsse)/12,4),False 

In [125]:
def function2(input_data_points,pred_day):
    test_range = all(ele <= 1941 for ele in pred_day) 
    if test_range:
        pred_day.sort()    
        raw_input_data = {'id':input_data_points,'item_id':[],'dept_id':[],'cat_id':[],'store_id':[],'state_id':[]}
        for data_point in input_data_points:
            data_point = data_point.split('_')
            raw_input_data['item_id'].append(f'{data_point[0]}_{data_point[1]}_{data_point[2]}')
            raw_input_data['dept_id'].append(f'{data_point[0]}_{data_point[1]}')
            raw_input_data['cat_id'].append(f'{data_point[0]}')
            raw_input_data['store_id'].append(f'{data_point[3]}_{data_point[4]}')
            raw_input_data['state_id'].append(f'{data_point[3]}')


        sales_train_evaluation = pd.read_pickle(r"best_model/sales_train_evaluation.pkl")  #The properties dataset
        calendar = pd.read_pickle(r"best_model/calender.pkl")  #The properties dataset
        sell_prices = pd.read_pickle(r"best_model/sell_prices.pkl")  #The properties dataset

        if(len(days_to_predict)!=1):
            start_day,end_day = min(days_to_predict),max(days_to_predict)
        else:
            start_day,end_day = days_to_predict[0],days_to_predict[0]

        sales_train_evaluation_ip = pd.DataFrame.from_dict(raw_input_data)

        sales_train_evaluation_col = [column for column in sales_train_evaluation.columns if 'd_' not in column]

        day_range_to_predict = [f'd_{day}' for day in range(start_day,end_day+1)]

        test_day = [f'd_{day}' for day in range(1942,1970)]

        d_cols = []
        for d_num in range(start_day - 90,start_day):
            d_cols.append(f'd_{d_num}')
        d_cols = d_cols + day_range_to_predict
        days_from_test_data = list(set(d_cols).intersection(test_day))

        for d in days_from_test_data:
            sales_train_evaluation_ip[d] = 0
            sales_train_evaluation_ip[d] = sales_train_evaluation_ip[d].astype(np.int16)

        sales_train_evaluation_ip_cols_filled = [d for d in sales_train_evaluation_ip.columns if 'd_' in d]
        days_from_rest = set(sales_train_evaluation_ip_cols_filled) ^ set(d_cols)
        items_to_select = sales_train_evaluation_ip[['id']]
        sales_train_selected_item = pd.merge(sales_train_evaluation,items_to_select,on='id',how='inner')
        sales_train_evaluation_d_columns = sales_train_selected_item[days_from_rest]
        sales_train_evaluation_ip = pd.concat([sales_train_evaluation_ip,sales_train_evaluation_d_columns],axis=1)
        sales_train_evaluation_ip_cols_cat,sales_train_evaluation_ip_cols_d = sales_train_evaluation_col,[d for d in list(sales_train_evaluation_ip.columns) if 'd_' in d]
        sales_train_evaluation_ip = sales_train_evaluation_ip[sales_train_evaluation_ip_cols_cat + list(np.sort(np.asarray(sales_train_evaluation_ip_cols_d)))]

        day_cols_calender = [column for column in sales_train_evaluation_ip.columns if 'd_' in column]
        calendar_ip = calendar[calendar['d'].isin(day_cols_calender)]
        sell_prices_ip = sell_prices

        roll_mat_csr_store = level_aggregator(sales_train_evaluation_ip)
        S = get_s(sales_train_evaluation_ip,pred_day,roll_mat_csr_store,drop_days=0)  
        stores_list = sales_train_evaluation_ip['store_id'].unique()

        df_ip = pd.melt(sales_train_evaluation_ip, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='units_sold').dropna()
        df_ip = pd.merge(df_ip, calendar_ip, on='d', how='left')
        df_ip = pd.merge(df_ip, sell_prices_ip, on=['store_id','item_id','wm_yr_wk'], how='left') 

        df_ip['date'] = pd.to_datetime(df_ip['date'])
        df_ip['day_of_month'] = df_ip['date'].dt.day.astype('int8')
        df_ip['week_of_month'] = df_ip['date'].apply(lambda d: (d.day-1) // 7 + 1).astype('int8')
        df_ip['week_of_year'] = df_ip['date'].dt.week.astype('int8')
        df_ip['quarter_of_year'] = df_ip['date'].dt.quarter.astype('int8')

        df_ip['is_month_start'] = [1 if i == 1 else 0 for i in df_ip['week_of_month']]
        df_ip['is_month_end'] = [1 if i in [4,5] else 0 for i in df_ip['week_of_month']]
        df_ip['event_name_1'] = df_ip['event_name_1'].fillna(0)
        df_ip['event_name_2'] = df_ip['event_name_2'].fillna(0)
        df_ip['event_type_1'] = df_ip['event_type_1'].fillna(0)
        df_ip['event_type_2'] = df_ip['event_type_2'].fillna(0)
        df_ip['is_week_end'] = [1 if i in [1,2] else 0 for i in df_ip['wday']]
        df_ip['close_to_week_days'] = [1 if i in [3,7] else 0 for i in df_ip['wday']]
        df_ip['mid_weeK_days'] = [1 if i in [4,5,6] else 0 for i in df_ip['wday']]
        df_ip['is_weekday'] = [1 if i not in [1,2] else 0 for i in df_ip['wday']]

        df_ip = df_ip.drop(['date','wm_yr_wk','weekday'],axis=1)

        avg_store_item_df = pd.read_pickle('best_model/preprocessed/avg_store_item_df_unique.pkl')
        avg_store_item_df = avg_store_item_df.drop_duplicates()

        df_ip = pd.merge(df_ip,avg_store_item_df,left_on=['store_id','item_id'],right_on=['store_id','item_id'])

        item_id_le = joblib.load('best_model/preprocessed/item_id_le.pkl')
        dept_id_le = joblib.load('best_model/preprocessed/dept_id_le.pkl')
        cat_id_le = joblib.load('best_model/preprocessed/cat_id_le.pkl')
        state_id_le = joblib.load('best_model/preprocessed/state_id_le.pkl')
        store_id_le = joblib.load('best_model/preprocessed/store_id_le.pkl')
        event_name_1_le = joblib.load('best_model/preprocessed/event_name_1_le.pkl')
        event_name_2_le = joblib.load('best_model/preprocessed/event_name_2_le.pkl')
        event_type_1_le = joblib.load('best_model/preprocessed/event_type_1_le.pkl')
        event_type_2_le = joblib.load('best_model/preprocessed/event_type_2_le.pkl')

        df_ip['item_id'] = item_id_le.transform(df_ip['item_id'].values)
        df_ip['dept_id'] = dept_id_le.transform(df_ip['dept_id'].values)
        df_ip['cat_id'] = cat_id_le.transform(df_ip['cat_id'].values)
        df_ip['state_id'] = state_id_le.transform(df_ip['state_id'].values)
        df_ip['store_id'] = store_id_le.transform(df_ip['store_id'].values)

        cat_df_ip = reduce_mem_usage(df_ip[['item_id','dept_id','cat_id','state_id','store_id']])
        df_ip = df_ip.drop(['item_id','dept_id','cat_id','state_id','store_id'],axis=1)

        df_ip['event_name_1'] = event_name_1_le.transform(df_ip['event_name_1'].astype(str).values)
        df_ip['event_name_2'] = event_name_2_le.transform(df_ip['event_name_2'].astype(str).values)
        df_ip['event_type_1'] = event_type_1_le.transform(df_ip['event_type_1'].astype(str).values)
        df_ip['event_type_2'] = event_type_2_le.transform(df_ip['event_type_2'].astype(str).values)

        event_df_ip = reduce_mem_usage(df_ip[['event_name_1','event_name_2','event_type_1','event_type_2']])
        df_ip = df_ip.drop(['event_name_1','event_name_2','event_type_1','event_type_2'],axis=1)
        df_test_ip = pd.concat([cat_df_ip[0],event_df_ip[0]],axis=1) 
        df_ip = pd.concat([df_ip,df_test_ip],axis=1)
        del cat_df_ip
        del event_df_ip
        del df_test_ip
        gc.collect()

        for lag_day,lag_column in zip([7,28,29,30,31,90],[f"lag_{lag}" for lag in [7,28,29,30,31,90]]):
            df_ip[lag_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].shift(lag_day).astype(np.float16)

        for rolling_day,rolling_column in zip([7,28,29,30,31,90],[f"rolling_{rolling}" for rolling in [7,28,29,30,31,90]]):
            df_ip[rolling_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].transform(lambda x: x.rolling(window=rolling_day).mean()).astype(np.float16)

        for rolling_day,rolling_column in zip([7,28,29,30,31,90],[f"rolling_std_{rolling}" for rolling in [7,28,29,30,31,90]]):
            df_ip[rolling_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].transform(lambda x: x.rolling(window=rolling_day).std()).astype(np.float16)

        for expanding_day,expanding_column in zip([7,28,29,30,31,90],[f"expanding_{expanding}" for expanding in [7,28,29,30,31,90]]):
            df_ip[expanding_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].transform(lambda x: x.expanding(expanding_day).mean()).astype(np.float16)

        for expanding_day,expanding_column in zip([7,28,29,30,31,90],[f"expanding_std_{expanding}" for expanding in [7,28,29,30,31,90]]):
            df_ip[expanding_column] = df_ip.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['units_sold'].transform(lambda x: x.expanding(expanding_day).std()).astype(np.float16)

        df_ip['d'] = df_ip['d'].apply(lambda x:x.split('_')[1]).astype(np.int16)
        df_ip['store_id'] = [str(i.split('_')[-3])+'_'+str(i.split('_')[-2]) for i in df_ip['id']]
        df_ip['units_sold'] = df_ip['units_sold'].fillna(0)

        with open('best_model/preprocessed/other_info.json') as f:
            other_info = json.load(f)

        df_ip['sell_price'] =  df_ip['sell_price'].fillna(float(other_info['median_sell_price']))
        df_ip['daily_avg_sales_unit'] = df_ip.groupby(['id','d'])['units_sold'].transform('mean').astype(np.float16)
        df_ip['weekly_avg_sales_unit'] = df_ip.groupby(['id','week_of_month'])['units_sold'].transform('mean').astype(np.float16)
        df_ip['sales_usd'] = df_ip['sell_price'] * df_ip['units_sold']

        w_weights_list = {}
        for store in stores_list:
            data = df_ip[df_ip['store_id'] == store]
            data = data[data['d']>=pred_day[0]-90]
            W = get_w(data[['id','sales_usd']],roll_mat_csr_store,store)
            w_weights_list[store] = W
            del data
            gc.collect()

        SW_store = {}
        for key in w_weights_list.keys():
            SW_store[key] = w_weights_list[key]/np.sqrt(S[key])

        SW_store1 = SW_store

        final_value = {}
        final_value['valid_pred'] = {}
        final_value['test_pred'] = {}
        remove_columns = ['id','store_id','sales_usd','daily_avg_sales_unit','is_weekday','units_sold']
        store_list = df_ip['store_id'].unique()
        for store in store_list:
    #         print(f"predicting for the store {store}")
            SW_store = SW_store1[store]
            df_store = df_ip[df_ip['store_id'] == store] 
            df_store = df_store[df_store['d'].isin(pred_day)]
            snap_feature  = ['snap_'+store.split('_')[0]]
            selected_columns = [column for column in df_store.columns if '7' not in column and 'rolling' not in column and 'snap' not in column and 'state' not in column]
            df_store = df_store[selected_columns+snap_feature]
            x_test,y_true = df_store.loc[(df_store['d']>=pred_day[0])].drop(remove_columns,axis=1),df_store[df_store['d']>=pred_day[0]]['units_sold'].values
            print('*****Prediction for Store: {}*****'.format(store))  
            dummy_model = joblib.load(f'best_model/gbdt/{store}_median_model1.pkl')
            test_pred_df = pd.DataFrame()
            #     val_score = np.sqrt(metrics.mean_squared_error(val_pred,y_valid))
            y_pred = dummy_model.predict(x_test)
            y_pred = np.asarray([round(val) for val in y_pred])
            num_of_days = len(pred_day)
            num_of_items = len(x_test['item_id'].unique())
            _,wrmsse_score,_ = wrmsse(y_pred,y_true,store,roll_mat_csr_store,SW_store,num_of_items,num_of_days)
            test_pred_df['id'] = df_store.loc[(df_store['d']>=pred_day[0])]['id']
            test_pred_df['store_id'] = df_store.loc[(df_store['d']>=pred_day[0])]['d']
            test_pred_df['units_sold'] = y_pred
            final_value['test_pred'][store] = test_pred_df

        dict_store_test = {}
        dict_store_test['id'] = []
        dict_store_test['d'] = []
        dict_store_test['units_sold'] = []
        for k,v in final_value['test_pred'].items():
            for i in v.values:
                dict_store_test['id'].append(i[0])
                dict_store_test['d'].append(i[1])
                dict_store_test['units_sold'].append(i[2]) 
        test_pred = pd.DataFrame.from_dict(dict_store_test)
    #     test_pred = pd.pivot(test_pred, index = 'id', columns = 'd', values = 'units_sold').reset_index()
        return wrmsse_score
    else:
        return False

In [126]:
# # if we want test with multiple products / days uncomment and test the following code
# start_time = time.time()
# input_data_points = ['FOODS_3_821_WI_3_evaluation','FOODS_3_822_WI_3_evaluation','FOODS_3_823_WI_3_evaluation']
# days_to_predict = [day for day in range(1913,1941)]
# wrmsse_score = function2(input_data_points,days_to_predict)
# print("running time {:.3f} secs...".format(time.time() - start_time))
# wrmsse_score

In [144]:
start_time = time.time()
input_data_points = ['FOODS_3_821_WI_3_evaluation']
days_to_predict = [1920]
wrmsse_score =  function2(input_data_points,days_to_predict)
if wrmsse_score:
    print(f"WRMSSE score is {wrmsse_score}")
else:
    print("We have data set with target values with in a range of 91 and 1941 so please give with in that range")
print("running time {:.3f} secs...".format(time.time() - start_time))

*****Prediction for Store: WI_3*****
WRMSSE score is 0.6061
running time 5.951 secs...


**Thank you**